In [1]:
import plotly.offline as pyo 
import plotly.graph_objs as go 
from plotly import tools
from plotly.subplots import make_subplots

import dash
from dash import html, dcc, dash_table, callback, Output, Input

import pandas as pd 
import sys 

sys.path.append('../')

import omdutils as omd

pd.options.display.max_columns = 999

In [2]:
# df = pd.read_csv('../Data/OldFaithful.csv')
# omd.to_sqlite(df, 'OldFaithful', '../SourceData/datasets.db')
sql = 'SELECT * FROM OldFaithful'
db = '../SourceData/datasets.db'
df = omd.from_sqlite(sql, db)
df.head()

,D,Y,X
0,1,78,4.4
1,1,74,3.9
2,1,68,4.0
3,1,76,4.0
4,1,80,3.5


In [3]:
data = [go.Scatter(x=df['X'], y=df['Y'], mode='markers', name='Old Faithful Eruptions')]    
layout = go.Layout(title='Old Faithful Eruptions', title_x=0.5, xaxis_title='Duration', yaxis_title='Interval')
fig = go.Figure(data=data, layout=layout)
fig.show()

In [4]:
eruption1 = 35
eruption1_f = f'{eruption1:.1f}'
eruption2 = 120
eruption2_f = f'{eruption2:.1f}'

In [5]:
app = dash.Dash(__name__)

def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

app.layout = html.Div(children=[
    html.Div(
        children='Eruption times of the Old Faithful geyser',
        style={'textAlign': 'center', 'color': '#111111', 'fontSize': 40},
        className='h1'
    ),
    html.Div(
        children='This is a simple dashboard',
        style={'textAlign': 'center', 'color': '#111111', 'fontSize': 30},
        className='h2'
    ),
    html.Div(
        children=[
            html.H2('Dashboard Information'),
            html.Table([
                html.Tr([
                    html.Td(
                        dcc.Markdown(f'''
                            ### This dashboard visualizes the duration times of the Old Faithful geyser by interval time.

                            **Eruption Details**:

                            - **Interval Between Eruptions**: Typically {eruption1_f} to {eruption2_f} minutes, averaging around 90 minutes.
                            - **Eruption Duration**: Lasts between 1.5 to 5 minutes.
                            - **Height of Eruptions**: Water can reach 106 to 185 feet (32 to 56 meters).
                            - **Factors Affecting Eruptions**: Seismic activity, seasonal changes, and water supply.

                            For real-time predictions, check the National Park Service's [Old Faithful Geyser predictions](https://www.nps.gov/yell/learn/photosmultimedia/webcams.htm).
                        '''),
                        style={'width': '70%', 'verticalAlign': 'top', 'paddingRight': '20px'}
                    ),
                    html.Td(
                        html.Img(
                            src='/assets/old_faithful.jpeg',  # Update this path as necessary
                            style={
                                'width': '100%',  # Set the width of the image
                                'borderRadius': '5px'  # Optional: Add rounded corners to the image
                            }
                        ),
                        style={'width': '30%'}
                    )
                ])
            ]),
        ],
        style={
            'backgroundColor': '#f0f0f0',  # Change this to your desired background color
            'padding': '20px',  # Add padding for better layout
            'borderRadius': '5px',  # Optional: Add rounded corners
            'boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.1)'  # Optional: Add a subtle shadow
        }
    ),
    dcc.Graph(figure=fig),
    dash_table.DataTable(data=df.to_dict('records'), page_size=10)
])

# Run the app
# if __name__ == '__main__':
app.run(debug=True)

In [8]:
# import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import os

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Directory to browse
BASE_DIRECTORY = os.getcwd()  # Change this to your desired directory

# Function to list files in a directory
def list_files_in_directory(directory):
    try:
        files = os.listdir(directory)
        return files
    except FileNotFoundError:
        return []

# Layout of the Dash app
app.layout = dbc.Container([
    html.H1("File Browser and Uploader"),
    
    # File upload component
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=True
    ),
    
    # Display uploaded files
    html.Div(id='output-file-upload'),
    
    # File explorer
    html.Hr(),
    html.H3("File Explorer"),
    dcc.Input(
        id='directory-input',
        type='text',
        value=BASE_DIRECTORY,
        style={'width': '100%'},
    ),
    html.Button('Browse', id='browse-button'),
    html.Ul(id='file-list')
])

# Callback to handle file uploads
@app.callback(
    Output('output-file-upload', 'children'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename'),
    State('upload-data', 'last_modified'),
)
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            html.Div([
                html.H5(name),
                # You can add more details or handling here if needed
            ]) for name in list_of_names
        ]
        return children

# Callback to update file list when browsing a directory
@app.callback(
    Output('file-list', 'children'),
    Input('browse-button', 'n_clicks'),
    State('directory-input', 'value')
)
def update_file_list(n_clicks, directory):
    if n_clicks is None:
        raise dash.exceptions.PreventUpdate

    files = list_files_in_directory(directory)
    if not files:
        return [html.Li("No files found or directory does not exist.")]

    return [html.Li(file) for file in files]

# Run the app
# if __name__ == '__main__':
app.run_server(debug=True)

In [6]:
import dash
from dash import dcc, html, Input, Output, State
import os

# Initialize the Dash app
app = dash.Dash(__name__)

# Directory to browse
BASE_DIRECTORY = os.getcwd()  # Change this to your desired directory

# Function to list files in a directory
def list_files_in_directory(directory):
    try:
        files = os.listdir(directory)
        return files
    except FileNotFoundError:
        return []

# Layout of the Dash app
app.layout = html.Div([
    html.H1("File Browser and Uploader"),
    
    # File upload component
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=True
    ),
    
    # Display uploaded files
    html.Div(id='output-file-upload'),
    
    # File explorer
    html.Hr(),
    html.H3("File Explorer"),
    dcc.Input(
        id='directory-input',
        type='text',
        value=BASE_DIRECTORY,
        style={'width': '100%'},
    ),
    html.Button('Browse', id='browse-button'),
    html.Ul(id='file-list')
])

# Callback to handle file uploads
@app.callback(
    Output('output-file-upload', 'children'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename'),
    State('upload-data', 'last_modified'),
)
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            html.Div([
                html.H5(name),
                # You can add more details or handling here if needed
            ]) for name in list_of_names
        ]
        return children

# Callback to update file list when browsing a directory
@app.callback(
    Output('file-list', 'children'),
    Input('browse-button', 'n_clicks'),
    State('directory-input', 'value')
)
def update_file_list(n_clicks, directory):
    if n_clicks is None:
        raise dash.exceptions.PreventUpdate

    files = list_files_in_directory(directory)
    if not files:
        return [html.Li("No files found or directory does not exist.")]

    return [html.Li(file) for file in files]

# Run the app
# if __name__ == '__main__':
app.run_server(debug=True)

In [7]:
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import os

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Directory to browse
BASE_DIRECTORY = os.getcwd()  # Change this to your desired directory

# Function to list files in a directory
def list_files_in_directory(directory):
    try:
        files = os.listdir(directory)
        return files
    except FileNotFoundError:
        return []

# Layout of the Dash app
app.layout = dbc.Container([
    html.H1("File Browser and Uploader"),
    
    # File upload component
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=True
    ),
    
    # Display uploaded files
    html.Div(id='output-file-upload'),
    
    # File explorer
    html.Hr(),
    html.H3("File Explorer"),
    dcc.Input(
        id='directory-input',
        type='text',
        value=BASE_DIRECTORY,
        style={'width': '100%'},
    ),
    html.Button('Browse', id='browse-button'),
    html.Ul(id='file-list')
])

# Callback to handle file uploads
@app.callback(
    Output('output-file-upload', 'children'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename'),
    State('upload-data', 'last_modified'),
)
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            html.Div([
                html.H5(name),
                # You can add more details or handling here if needed
            ]) for name in list_of_names
        ]
        return children

# Callback to update file list when browsing a directory
@app.callback(
    Output('file-list', 'children'),
    Input('browse-button', 'n_clicks'),
    State('directory-input', 'value')
)
def update_file_list(n_clicks, directory):
    if n_clicks is None:
        raise dash.exceptions.PreventUpdate

    files = list_files_in_directory(directory)
    if not files:
        return [html.Li("No files found or directory does not exist.")]

    return [html.Li(file) for file in files]

# Run the app
# if __name__ == '__main__':
app.run_server(debug=True)

In [16]:
from dash import Dash, dcc, html, dash_table, Input, Output, State, callback

import base64
import datetime
import io

import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload'),
])

def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            df.to_dict('records'),
            [{'name': i, 'id': i} for i in df.columns]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])

@callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children

# if __name__ == '__main__':
app.run(debug=True)

In [13]:
app = dash.Dash(__name__)

markdown_text = '''
    ### Dash and Markdown 
    Dash apps can be written in Markdown. 
    Dash uses the [CommnMark](http://commonmark.org/) specification of Markdown.  
    Check out their [60 Second Markdown Tutorial](http://commonmark.org/help/)
    if this is your first introduction to Markdown!  
    Markdown includes syntax for things like **bold text** and *italics*, 
    [links](htt://commonmark.org/help), inline `code` snippets, lists, quotes, and more.
    '''

app.layout = html.Div([
    dcc.Markdown(children=markdown_text),
    html.Div([html.Label('Dropdown'),
    dcc.Dropdown(options=[{'label': 'New York City', 'value': 'NYC'}, 
                          {'label': 'San Francisco', 'value':'SF'}],
                           value='SF')],),
    html.P(),
    html.Div([html.Label('Slider'),
    dcc.Slider(min=-10, max=10, step=1.0, value=0)]),

    html.P(),
    html.Div([html.Label('Some Radio Items'),
    dcc.RadioItems(options=[{'label': 'New York City', 'value': 'NYC'},
                            {'label': 'San Francisco', 'value': 'SF'}],
                            value='SF')])
               ])

app.run(debug=True)

Error: no such file, variable, URL, history range or macro
